In [31]:
import numpy as np

#Read coordinate csv. Col 1 is x coords and Col2 is y coords.
def read_coords(filename):    
    myFile = open(filename)
    row =0
    coords =[]
    for line in myFile:
        coords.append(line.rstrip().split(",")[:])
        #coords[row] = line.rstrip().split(",")[:]
    myFile.close()
    return coords
#Find the sum of x to a certain power. This is used to construct the terms of the left hand side of the system
def findSum(coords,power):
    sumOfCoords = 0
    for point in coords:
        sumOfCoords += float(point[0])**power
    return sumOfCoords
#Find the sum of x*y where x is raised to a power. This is used to determine terms on the RHS of system
def findEndTerm(coords, power):
    sumOfCoords = 0
    for point in coords:
        sumOfCoords += (float(point[0])**power)*float(point[1])
    return sumOfCoords

'''
System looks as follows:
(sum[x^0])*a0 + (sum[x^1])*a1 + ... + (sum[x^n])*an = sum[x^m * y]
(sum[x^1])*a0 + (sum[x^2])*a1 + ... + (sum[x^n])*an = sum[x^m * y]
(sum[x^2])*a0 + (sum[x^3])*a1 + ... + (sum[x^n])*an = sum[x^m * y]

Notice that the first term in each row, is always to the power of the zero-indexed row number, so
in row 1 ( row 0 ) we start with 0, and pattern remains, so we will start iteraing from `rowNumber`
Then it goes up to rowNumber + dimension, so if our data is a 3x3 square matrix, dimension is 3 so on each line
we iterate up to rowNumber + 3.
The power term is then always raised to the same power as the first term, so the index of the current row. 

Using these rules we can form our system:
'''
def formSystem(coords, dim):
    #Define system arr
    system = []
    
    for row in range(dim):
        #Define current row, that will contain column data as its entries
        temp = []
        #We iterate up to row number + dimension on each row, +1 for inclusivity
        for col in range(row, row+dim+1):
            #If this isnt the last row then find the sum term
            if(col != row+dim):
                sumToPow = findSum(coords, col)
                temp.append(sumToPow)
            #If got to last term, calculate that
            else:
                lastTerm = findEndTerm(coords, row)
                temp.append(lastTerm)
        #Add row
        system.append(temp)
    return system

#Wrapper for regression
def pol_regression(features_train, y_train, degree):
    system = formSystem(features_train, degree+1)
    #np.linalg.solve(system)
    
    #Convert to np array
    system = np.asarray(system)
    #.shape[1] is how many columns are in each row
    cols = system.shape[1]
    #keep all but the last row (RHS)
    a = system[:,0:cols-1]
    #Right hand side is then 
    b = system[:,cols-1]
    #print(a)
    #print(b)
    #print(system)
    solution = np.linalg.solve(a, b)
    return system, solution

data = read_coords("task1.csv")
#print(data)
system,solution = pol_regression(data, data, 2)
print(system)
print(solution)



[[    20.            -13.86328553    146.05154296   -608.60115461]
 [   -13.86328553    146.05154296   -272.63897143   2455.25640845]
 [   146.05154296   -272.63897143   2273.40387611 -10947.41660878]]
[ 5.514104    9.9002525  -3.98238317]
